In [10]:
import numpy as np
import torch
import os
import h5py
import pickle
import fnmatch
import cv2
import sys
from time import time
from torch.utils.data import TensorDataset, DataLoader
from scipy.spatial.transform import Rotation

np.set_printoptions(suppress=True)


In [11]:
act_path = "/home/zfei/code/act-plus-plus/"
sys.path.append(act_path)
from utils import EpisodicDataset
from utils import find_all_hdf5, flatten_list, get_norm_stats

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
dataset_path_list = find_all_hdf5("/mnt/data1/act/train_act_scara_3cam/data1/", skip_mirrored_data=False)
norm_stats, all_episode_len = get_norm_stats(dataset_path_list)

camera_names = ['wrist', 'wrist_down', 'top']
train_episode_ids = [0, 1]
train_episode_len = [all_episode_len[i] for i in train_episode_ids]

chunk_size = 64
policy_class = "EPACT"

Found 51 hdf5 files


In [13]:
train_dataset = EpisodicDataset(dataset_path_list, 
                                camera_names, 
                                norm_stats, 
                                train_episode_ids, 
                                train_episode_len, 
                                chunk_size, 
                                policy_class)


URDF Path:  /home/zfei/code/act-plus-plus/urdf/hitbot_model.urdf
kdl_parse urdf ok?:  True
augment_images:  False
Initializing transformations


In [14]:
def find_all_datafolder(dataset_dir):
    datafolders = []
    for root, dirs, files in os.walk(dataset_dir, followlinks=True):
        for dirname in fnmatch.filter(dirs, 'seg_*'):
            datafolders.append(os.path.join(root, dirname))
    print(f'Found {len(datafolders)} datafolders under {dataset_dir}')
    return datafolders

def get_norm_stats_folder(dataset_path_list):
    all_pose_data = []
    all_grip_data = []
    all_episode_len = []

    for dataset_path in dataset_path_list:
        try:
            action = np.loadtxt(dataset_path + "/gripper_distances.txt").reshape(-1, 1)
            camera_trajs = np.loadtxt(dataset_path + "/CameraTrajectory.txt")
            camera_trajs_mat = camera_trajs.reshape(-1, 3, 4)
            pose = np.zeros((len(camera_trajs), 6))
            pose[:, :3] = camera_trajs_mat[:, :3, 3]
            euler_angles = rotation_matrix_to_euler_angles(camera_trajs_mat[:, :3, :3])
            pose[:, 3:] = euler_angles
        except Exception as e:
            print(f'Error loading {dataset_path} in get_norm_stats')
            print(e)
            quit()
        all_pose_data.append(torch.from_numpy(pose))
        all_grip_data.append(torch.from_numpy(action))
        all_episode_len.append(len(action))
    all_pose_data = torch.cat(all_pose_data, dim=0)
    all_grip_data = torch.cat(all_grip_data, dim=0)
    # cat pose and grip data
    all_action_data = torch.cat([all_pose_data, all_grip_data], dim=1)

    # normalize action data
    action_mean = all_action_data.mean(dim=[0]).float()
    action_std = all_action_data.std(dim=[0]).float()
    action_std = torch.clip(action_std, 1e-2, np.inf) # clipping
    action_min = all_action_data.min(dim=0).values.float()
    action_max = all_action_data.max(dim=0).values.float()

    eps = 0.0001
    stats = {"action_mean": action_mean.numpy(), "action_std": action_std.numpy(),
             "action_min": action_min.numpy() - eps,"action_max": action_max.numpy() + eps}

    return stats, all_episode_len

def rotation_matrix_to_euler_angles(rotation_matrices):
    """Converts a batch of rotation matrices to Euler angles (roll, pitch, yaw).

    Args:
    rotation_matrices: A list of 3x3 rotation matrices.

    Returns:
    A list of Euler angles (roll, pitch, yaw) corresponding to each rotation matrix.
    """

    rotations = Rotation.from_matrix(rotation_matrices)
    euler_angles = rotations.as_euler('xyz', degrees=False)
    return euler_angles

In [15]:
dataset_path_list = find_all_datafolder("/home/zfei/data/UPI/lab_sb/rs435i_lab_picking_024-08-29-09-07-12/grip_data_seg/")
norm_stats, all_episode_len  = get_norm_stats_folder(dataset_path_list)

camera_names = ['color']
train_episode_ids = [0, 1]
train_episode_len = [all_episode_len[i] for i in train_episode_ids]

chunk_size = 64
policy_class = "ACTEP"
policy_class = "ACT"



Found 41 datafolders under /home/zfei/data/UPI/lab_sb/rs435i_lab_picking_024-08-29-09-07-12/grip_data_seg/


In [37]:
from datasets import EpisodicDatasetRs435i

In [40]:
dataset = EpisodicDatasetRs435i(dataset_path_list, camera_names, norm_stats, train_episode_ids, train_episode_len, chunk_size, policy_class)

augment_images:  False
episode_id:  0
start_ts:  0
Initializing transformations


In [42]:
image_data, action_data, is_pad = dataset[50]

episode_id:  0
start_ts:  50


In [43]:
print(image_data.shape)
print(action_data.shape)
print(is_pad)

torch.Size([1, 3, 480, 848])
torch.Size([64, 7])
tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True])
